<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li></ul></div>

# Проект для «Викишоп»

Интернет-магазин запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

Импортируем необходимые библиотеки

In [1]:
! pip install pymystem3

In [1]:
import pandas as pd
from pymystem3 import Mystem
import re
import nltk
from nltk.corpus import stopwords as nltk_stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score

Прочитаем файл

In [2]:
data = pd.read_csv('/datasets/toxic_comments.csv')

Посмотрим на наши данные

In [3]:
display(data.head())

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


Загрузим общую информацию о данных.

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159571 non-null  object
 1   toxic   159571 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


**Вывод**

Данные загружены, пропусков нет, типы данных верные.

Перейдем к подготовке данных для обучения. Для начала удалим стоп-слова и лемматизируем текст.

In [5]:
def lemmatize(text):
    m = Mystem()
    lemm_list = m.lemmatize(text)
    lemm_text = "".join(lemm_list)
        
    return clear_text(lemm_text)


def clear_text(text):
    text = " ".join(re.sub(r'[^a-zA-Z]', ' ', text).split())
    return text

data['lemm_text'] = data['text'].apply(clear_text)

display(data.head())

,text,toxic,lemm_text
0,Explanation\nWhy the edits made under my usern...,0,Explanation Why the edits made under my userna...
1,D'aww! He matches this background colour I'm s...,0,D aww He matches this background colour I m se...
2,"Hey man, I'm really not trying to edit war. It...",0,Hey man I m really not trying to edit war It s...
3,"""\nMore\nI can't make any real suggestions on ...",0,More I can t make any real suggestions on impr...
4,"You, sir, are my hero. Any chance you remember...",0,You sir are my hero Any chance you remember wh...


Создадим корпус

In [6]:
corpus = list(data['lemm_text'])

Создадим мешок слов и n-граммы.

In [7]:
nltk.download('stopwords')
stopwords = set(nltk_stopwords.words('english'))

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Выделим признаки и целевой признак, разобем выборку на обучающую и тестовую.

In [8]:
features = corpus
target = data['toxic']

features_train, features_valid, target_train, target_valid = train_test_split(
    features, target, test_size=0.20, random_state=12345)

count_vect = CountVectorizer(stop_words=stopwords, ngram_range=(1, 3))
features_train = count_vect.fit_transform(features_train)
features_valid = count_vect.transform(features_valid)
display(features_train.shape)
display(features_valid.shape)

(127656, 5829309)

(31915, 5829309)

## Обучение

Обучим логистическую регрессию, проверим на валидационной выброрке.

In [9]:
model = LogisticRegression(random_state=12345, solver='lbfgs')

model.fit(features_train, target_train)
predict = model.predict(features_train)

display(f'F1-мера на обучающей выборке: {f1_score(target_train, predict)}')

/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


'F1-мера на обучающей выборке: 0.9871555347968238'

In [10]:
predict_valid = model.predict(features_valid)
display(f'F1-мера на валидационной выборке: {f1_score(target_valid, predict_valid)}')

'F1-мера на валидационной выборке: 0.7620550587410135'

## Выводы

В данной работе исследовали алгоритм лемматизации текста, очистили текст от стоп-слов. После разбивки на обучающую и валидационную выборки провели векторизацию текста. Обучив логистическую регрессию, получили полне достойный результат F1-меры, равный 0,76. Так как целевая метрика достигнута - могу рекомендовать к использованию линейную регрессию - в виду наименьшего использования ресурсов.